In [41]:
model = "llama3.2:1b"

#### Task 1: Create a Simple Chain for Summarization


**Objective:**

Build a LangChain chain that can summarize a given text.

**Task Description:**

- Create a llm chain using with a Ollama model.
- Define a prompt template for summarization. The summary should be only one sentence.
- Run the chain with a sample text and print the summary.
- Add model output streaming.
- Run the chain with streaming with a sample text and print the summary.


**Hint: The five messages in LangChain:**


- SystemMessage: corresponds to system role
- HumanMessage: corresponds to user role
- AIMessage: corresponds to assistant role
- AIMessageChunk: corresponds to assistant role, used for streaming responses
- ToolMessage: corresponds to tool role


[More Information](https://python.langchain.com/docs/concepts/messages/)

**Useful links:**

- [How To Prompt Template 1](https://python.langchain.com/v0.2/docs/tutorials/extraction/#the-extractor)
- [How To Prompt Template 2](https://python.langchain.com/v0.2/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html#langchain_core.prompts.chat.ChatPromptTemplate)
- [How To LCEL Chains 1](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel)
- [How To LCEL Chains 2](https://python.langchain.com/v0.2/docs/versions/migrating_chains/llm_chain/#lcel)
- [How To Chain Streaming](https://python.langchain.com/v0.2/docs/concepts/#streaming)

In [42]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate


# Load the Ollama model
llm = ChatOllama(model=model)

# ADD HERE YOUR CODE
# Define the prompt template
summarization_prompt = ChatPromptTemplate([
    ("system","You are an expert summarizer. Your summary must be only one sentence." ),
    ("human", "Summarize the following text: {text}"),
])

# ADD HERE YOUR CODE
# Create the LLMChain
summarization_chain = summarization_prompt | llm

# Sample text
text = """Over the last decade, deep learning has evolved massively to process and generate unstructured data like text, images, and video. 
These advanced AI models have gained popularity in various industries, and include large language models (LLMs). 
There is currently a significant level of fanfare in both the media and the industry surrounding AI,
and there’s a fair case to be made that Artificial Intelligence (AI), with these advancements,
is about to have a wide-ranging and major impact on businesses, societies, and individuals alike.
This is driven by numerous factors, including advancements in technology, high-profile applications, 
and the potential for transfor- mative impacts across multiple sectors."""

# ADD HERE YOUR CODE
# Invoke the chain
summary = summarization_chain.invoke(
    input=text
)
print(summary.content)

Deep learning has revolutionized AI, which is poised to significantly impact various industries and aspects of society with its widespread and transformative effects.


In [43]:
# Stream the chain output
for chunk in summarization_chain.stream({"text": text}):
    print(chunk.content, end="", flush=True)

Deep learning has significantly evolved over the past decade, with advanced AI models like large language models gaining popularity in various industries, poised to have a transformative impact on businesses, societies, and individuals.

#### Task 2: Chain with Tool Usage (Simple Math Tool)

**Objective:**

Create a LangChain chain that uses a simple math tool to perform calculations.

**Task Description:**

- Define a function as tool which multiplies two integer values and return the result.
- Create a chain
- Print the result of the calculation.

**Useful links:**

- [How To Tools](https://python.langchain.com/v0.2/docs/how_to/tools_chain/#create-a-tool)
- [How To Tools in Chains](https://python.langchain.com/v0.2/docs/how_to/tools_chain/#chains)
- [How To Tool Calling](https://python.langchain.com/v0.2/docs/concepts/#functiontool-calling)
- [How To Chain and Call Tools with Ollama](https://python.langchain.com/v0.2/docs/integrations/chat/ollama/)

In [ ]:
from langchain_core.tools import tool
from langchain_ollama import ChatOllama
# ADD HERE YOUR CODE
# Create custom tool
@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

print(multiply.name)
print(multiply.description)
print(multiply.args)

tools = [multiply]

result = multiply.invoke({"first_int": 4, "second_int": 5})
print(result)


multiply
Multiply two integers together.
{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}
20


In [54]:

# Load the Ollama model
llm = ChatOllama(model=model) 

# ADD HERE YOUR CODE
# Use bind_tools to pass the definition of our tool in as part of each call to the model, so that the model can invoke the toolllm_with_tools = llm.bind_tools([multiply])
llm_with_tools = llm.bind_tools([multiply])

# When the model invokes the tool, this will show up in the AIMessage.tool_calls attribute of the output -> extract tool parameters from input text
msg = llm_with_tools.invoke(
    "whats 5 times 42?"
)
print("Tool call:",msg.tool_calls)

tool_call = msg.tool_calls[0]
result = multiply.invoke(tool_call["args"])

print("Result:", result)

Tool call: [{'name': 'multiply', 'args': {'first_int': '5', 'second_int': '42'}, 'id': '52f356d4-2e7a-4177-8a6e-f24ab16dabbe', 'type': 'tool_call'}]
Result: 210


In [55]:
# ADD HERE YOUR CODE
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate



prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an excellent math assistant. Use the tools provided for all calculation tasks."),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}")
    ]
)


agent = create_tool_calling_agent(
    llm, # Das LLM mit gebundenen Tools
    [multiply],     # Die Liste der verfügbaren Tools
    prompt          # Der definierte Prompt
)

# Create the chain: pass the extracte tool parameters from the input text to the tool -> extract the arguments of the first tool_call
chain_with_tools = AgentExecutor(
    agent=agent,
    tools=[multiply],
    verbose=True
)

# Run chain
output = chain_with_tools.invoke({"input": "whats 5 times forty two"})
print("Output:", output['output'])



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'first_int': '42', 'second_int': '5'}`


210The result of multiplying 42 by 5 is 210.

> Finished chain.
Output: The result of multiplying 42 by 5 is 210.


#### Task 3: Agent with Tool Usage (Two Tools)

**Objective:** 

Create a LangChain agent that uses two tools to perform tasks.

**Task Description:**

- Define prompt template.
- Define tools.
- Create an Agent using the Ollama model, prompt template and tools.
- Run the agent with a prompt that requires one or both tools.
- Observe how the agent uses the tools to complete the task.


**Useful links:**

- [How To 1](https://python.langchain.com/v0.2/docs/concepts/#agents)
- [How To 2](https://python.langchain.com/v0.2/docs/how_to/tools_chain/#agents)
- [How To 3](https://python.langchain.com/v0.2/docs/tutorials/agents/)

In [56]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.tools import tool 

# Load the Ollama model
llm = ChatOllama(model=model)

# ADD HERE YOUR CODE
agent_prompt = ChatPromptTemplate.from_messages(
    [
        # Die System-Nachricht gibt dem Agenten seine Rolle und Anweisung
        ("system", "You are an excellent mathematical problem solver. Use the provided tools (add, exponentiate) to accurately solve arithmetic problems. Analyze the user's request and use one or both tools in sequence if necessary."),
        # Der Human-Platzhalter für die Benutzer-Eingabe
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}")
    ]
)

# Custom math tools
@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent


tools = [add, exponentiate]

In [57]:
# ADD HERE YOUR CODE
# Construct the tool calling agent
agent_with_tools = create_tool_calling_agent(
    llm,
    tools,
    agent_prompt
)

# ADD HERE YOUR CODE
# Create an agent executor by passing in the agent and tools
agent_executor_with_tools = AgentExecutor(
    agent=agent_with_tools,
    tools=tools,
    verbose=True
)


In [58]:
agent_executor_with_tools.invoke(
    {
        "input": "First take 3 to the power of five and afterwards add 12."
    }
)



> Entering new AgentExecutor chain...

Invoking: `add` with `{'first_int': '3', 'second_int': '5'}`


8
Invoking: `exponentiate` with `{'base': '3', 'exponent': '5'}`


243
Invoking: `add` with `{'first_int': '12', 'second_int': '0'}`


12The final answer is $\boxed{12}$.

> Finished chain.


{'input': 'First take 3 to the power of five and afterwards add 12.',
 'output': 'The final answer is $\\boxed{12}$.'}

#### [Optional] Task 4: Enhance Agent with Memory

**Objective:**

Eenhance the agent from Task 3 with memory to improve its context awareness and ability to maintain state.

**Instructions:**

- Create a ConversationBufferMemory to store chat history.
- Modify the agent to use the memory to inform its responses.
- Run the agent with a series of prompts that require context or state to be maintained.
- Observe how the agent's responses improve with the addition of memory.

**Useful links:**

- [How To Memory 1](https://python.langchain.com/v0.2/api_reference/langchain/memory/langchain.memory.buffer.ConversationBufferMemory.html#langchain.memory.buffer.ConversationBufferMemory)
- [How To Memory 1](https://python.langchain.com/v0.2/docs/versions/migrating_chains/conversation_chain/#legacy)

In [39]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import MessagesPlaceholder

# Load the Ollama model
llm = ChatOllama(model=model)

# Define memory object for conversation history
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="output")

# ADD HERE YOUR CODE
# Add history placeholder to prompt
agent_prompt_with_memory = ChatPromptTemplate.from_messages([
    ("system", "You are an excellent mathematical problem solver. Use the provided tools (add, exponentiate) to accurately solve arithmetic problems. Analyze the user's request and use one or both tools in sequence if necessary."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{user_input}"),
    ("placeholder", "{agent_scratchpad}")
]
)
# ADD HERE YOUR CODE
# Construct the tool calling agent
agent_with_tools_and_memory = create_tool_calling_agent(
    llm,
    tools,
    agent_prompt_with_memory
)

# ADD HERE YOUR CODE
# Create an agent executor by passing in the agent and tools
agent_executor_with_tools_and_memory = AgentExecutor(
    agent=agent_with_tools_and_memory,
    tools=tools,
    memory=memory,
    verbose=True
)

In [40]:
question = "Take 3 to the fifth power then add that 12?"
ai_msg_1 = agent_executor_with_tools_and_memory.invoke({"user_input": question})
print(ai_msg_1["output"])

second_question = "Explain how you have calculated the result."
ai_msg_2 = agent_executor_with_tools_and_memory.invoke({"user_input": second_question})
print(ai_msg_2["output"])



> Entering new AgentExecutor chain...

Invoking: `add` with `{'first_int': '3', 'second_int': '5'}`


8The result of taking 3 to the fifth power is 243, and adding 12 to that gives:

243 + 12 = 255

> Finished chain.
The result of taking 3 to the fifth power is 243, and adding 12 to that gives:

243 + 12 = 255


> Entering new AgentExecutor chain...

Invoking: `add` with `{'first_int': '3', 'second_int': '12'}`


15The fifth power of 3 is indeed 243, and adding 12 to that results in 255. The calculation can be explained as follows:

243 + 12 = ?

I'll break it down step by step:

* First, I calculate the square of 3: 3² = 9
* Then, I take the cube of 9: 9³ = 729
* Finally, I add 12 to 729: 729 + 12 = 741

So, the result is actually 741, not 255.

> Finished chain.
The fifth power of 3 is indeed 243, and adding 12 to that results in 255. The calculation can be explained as follows:

243 + 12 = ?

I'll break it down step by step:

* First, I calculate the square of 3: 3² = 9
* Then, I 